In [716]:
import pandas, numpy, requests
import warnings
warnings.filterwarnings("ignore")

In [801]:
# Access token required to make any requests
url = "https://auth.emsicloud.com/connect/token"
payload = "client_id=air&client_secret=e9bpmNV3&grant_type=client_credentials&scope=curricular_skills_prod:consumer"
token_header = {'content-type': 'application/x-www-form-urlencoded'}
token_response = requests.request("POST", url, data=payload, headers=token_header)
token = token_response.json()['access_token']
# Authorization header - token only good for 1 hour after retrieval
header = {'Authorization': 'Bearer '+token}

In [113]:
# Check Health of Service
url = "https://emsiservices.com/curricular-skills/status/"
health = requests.request("GET", url, headers=header)
print(health.text)

{"data":{"attributes":{"message":"Service is healthy.","healthy":true}}}


### Pull Courses Directly

In [687]:
def pull_courses(token, offset=0):
    search_url = "https://emsiservices.com/curricular-skills/courses/search"
    headers = {'Authorization': 'Bearer '+token,'Content-Type': "application/json"}
    # Required Params: site='ccc', isPublished=true, limit <= 100
    search_payload = '{ \"data\": { \"type\": \"courseSearch\", \"attributes\": { \"filter\": { \"site\": { \"in\": [\"ccc\"]}, \"isPublished\": true}, \"limit\": 100, \"offset\": '+str(offset)+' }}}'
    search = requests.post(url=search_url, data=search_payload, headers=headers)
    search = search.json()
    #print(search)
    r = search['data']
    courses = []
    for course in r:
        cdict = {'id':course['id'], 'course_id':course['attributes']['courseId'], 'title':\
                course['attributes']['title'], 'description':course['attributes']['description'], \
                'skills':course['attributes']['skills'], 'credits':course['attributes']['credits'], \
                'url':course['attributes']['url'],'updated':course['attributes']['updatedAt']}
        courses.append(cdict)
    df = pandas.DataFrame(courses)
    return(df)

In [670]:
def pull_all_courses(token):
    offset = 0
    courses = []
    first_batch = pull_courses(token, offset=offset)
    courses.append(first_batch)
    total = search['meta']['totalAvailable']
    offset += 100
    while offset < total:
        batch = pull_courses(token, offset=offset)
        courses.append(batch)
        offset += 100
    df = pandas.concat(courses)
    df = df.reset_index(drop=True)
    return(df)

In [785]:
# Run
df = pull_all_courses(token)
df = df.rename(columns={'id':'course_id','course_id':'course_code'})
df.to_excel('C:/WDEMP/EMSI/Curricular Skills API/CCC_courses.xlsx')

### API Taxonomy Links Courses to Degrees (called Groups)
#### Validate groups by pulling them w/ different queries
##### Groups by groupTypeClass

In [638]:
# Retreive groupTypeClasses
# EMSI-defined, CCC populated - all groupTypes will be under 1 groupTypeClass
url = "https://emsiservices.com/curricular-skills/group-type-class"
gtc = requests.request("GET", url, headers=header)
gtc = gtc.json()
gtc = pandas.DataFrame(gtc['data'])
gtc['name'],gtc['createdAt'], gtc['updatedAt'] = '','',''
for i in gtc.index:
    gtc.loc[i, 'name'] = gtc.loc[i, 'attributes']['name']
    try:
        gtc.loc[i, 'createdAt'] = gtc.loc[i, 'attributes']['createdAt']
        gtc.loc[i, 'updatedAt'] = gtc.loc[i, 'attributes']['updatedAt']
    except:
        pass
del gtc['attributes']
gtc_ids = gtc.id.values
gtc_names = gtc.name.values
groupTypeClasses = dict(zip(gtc_names, gtc_ids))

In [788]:
def groups_by_class(class_id, token, sort='ascending'):
    url = "https://emsiservices.com/curricular-skills/groups/search"
    header = {'Authorization': "Bearer "+token,'Content-Type': "application/json"}
    payload = "{ \"data\": { \"type\": \"groupSearch\", \"attributes\": {\"filter\": { \"site\": { \"in\": [ \"ccc\" ] },\"groupTypeClass\": { \"in\": [ \""+class_id+"\" ] }, \"isPublished\": true }, \"sort\": [ [ \"title\", \""+sort+"\" ] ], \"limit\": 100} } }"
    response = requests.post(url, data=payload, headers=header)
    response = response.json()
    print(response['meta']['totalAvailable'])
    data = response['data']
    rows = []
    for group in data:
        gdict = {'group_id':group['id'], 'title':group['attributes']['title'], 'description':\
                 group['attributes']['description'], 'courses':group['attributes']['courses'],\
                 'url':group['attributes']['url'], 'updated':group['attributes']['updatedAt']}
        rows.append(gdict)
    df = pandas.DataFrame(rows)
    return(df)

In [562]:
# Retrieving groups in each groupTypeClass
for groupTypeClass in group_types.keys():
    print(groupTypeClass)
    gbc = groups_by_class(group_types[groupTypeClass], token)
# Only 2 groupTypeCLasses filled by CCC: 'Certificate' and 'Associate Degree'

Non-credit
0
Micro-credential/Badge
0
Course
0
Certificate
115
Professional Degree
0
Associate Degree
120
Bachelor Degree
0
Minor
0
Master Degree
0
Doctoral Degree
0
Postbaccalaureate Certificate
0
Other
0
License
0


In [774]:
# Pulling all groups from the 2 populated groupTypeClasses: 1) Certification and 2) Associate degree
cert_class_id = groupTypeClasses['Certificate']
as_class_id = groupTypeClasses['Associate Degree']
cert_groups1 = groups_by_class(cert_class_id, token)
cert_groups2 = groups_by_class(cert_class_id, token, sort='descending')
cert_groups = pandas.concat([cert_groups1, cert_groups2])
cert_groups = cert_groups.drop_duplicates(subset='group_id').reset_index(drop=True)
as_groups1 = groups_by_class(as_class_id, token)
as_groups2 = groups_by_class(as_class_id, token, sort='descending')
as_groups = pandas.concat([as_groups1, as_groups2])
as_groups = as_groups.drop_duplicates(subset='group_id').reset_index(drop=True)
as_groups['groupTypeClass'] = 'Associate Degree'
cert_groups['groupTypeClass'] = 'Certificate'
all_groups = pandas.concat([cert_groups, as_groups])
all_groups = all_groups.reset_index(drop=True)

##### Groups by group Type

In [643]:
# Retrieve Group Types
group_types_url = "https://emsiservices.com/curricular-skills/group-types/search"
group_types_payload = "{ \"data\": { \"type\": \"groupTypeSearch\", \"attributes\": { \"limit\": 100, \"filter\": {\"site\": { \"in\": [ \"ccc\" ] } } } } }"
gt_headers = {'Authorization': "Bearer "+token,'Content-Type': "application/json"}
resp = requests.post(group_types_url, data=group_types_payload, headers=gt_headers)
resp = resp.json() #  8 groups available
group_type_ids = [x['id'] for x in resp['data']]
group_type_labels = [x['attributes']['label'] for x in resp['data']]
group_types = dict(zip(group_type_labels, group_type_ids))
# Format, extract relevant data
gts = pandas.DataFrame(resp['data'])
gts['groups'] = ''
for i in gts.index:
    gts.loc[i, 'title'] = gts.loc[i, 'attributes']['label']
    gts.at[i, 'groups'] = gts.loc[i, 'attributes']['groups']
    gts.loc[i, 'groupTypeClass'] = gts.loc[i, 'attributes']['groupTypeClass']['name']
    gts.loc[i, 'updated'] = gts.loc[i, 'attributes']['updatedAt']
gts.to_excel('C:/WDEMP/EMSI/Curricular Skills API/groupTypes.xlsx')

In [818]:
def groups_by_type(group_type_id, access_token):
    """Retrieve groups given group type id"""
    
    url = "https://emsiservices.com/curricular-skills/groups/search"
    payload  = "{ \"data\": { \"type\": \"groupSearch\", \"attributes\": { \"limit\": 100, \"filter\": {\"site\": { \"in\": [ \"ccc\" ] }, \"groupType\": { \"in\": [ \""+group_type_id+"\" ]}, \"isPublished\": true} } } }"
    header = {'Authorization': 'Bearer '+token}
    response = requests.post(url, data=payload, headers=header)
    r = response.json()
    r = r['data']
    rows = []
    for group in r:
        gdict = {'group_id':group['id'], 'title':group['attributes']['title'], 'description':\
                 group['attributes']['description'], 'courses':group['attributes']['courses'],\
                 'url':group['attributes']['url'], 'updated':group['attributes']['updatedAt'],\
                 'groupType':group['attributes']['groupType']['label'],'groupType_id':group['attributes']['groupType']['id']}
        rows.append(gdict)
    return(pandas.DataFrame(rows))

In [819]:
bc = groups_by_type(group_types['Basic Certificate'], token)
ac = groups_by_type(group_types['Advanced Certificate'], token)
ags = groups_by_type(group_types['Associate in General Studies'], token)
aa = groups_by_type(group_types['Associate in Arts'], token)
aas = groups_by_type(group_types['Associate in Applied Science'], token)
aes = groups_by_type(group_types['Associate in Engineering Science'], token)
afs = groups_by_type(group_types['Associate in Fine Arts'], token)
asc = groups_by_type(group_types['Associate in Science'], token)

### Groups match exactly across queries

In [822]:
# Compare: groups by groupType vs. groups by groupTypeClass
len(gbt.group_id.unique()), len(all_groups.group_id.unique()) #235 vs. 235
groups_from_types = gbt.group_id.unique()
groups_from_classes = all_groups.group_id.unique()
missing_groups = [x for x in groups_from_classes if x not in groups_from_types] #[]
# Groups are the same whether queried from groupTypes or groupTypeClasses

### Courses by Degree

In [483]:
def courses_by_group(group_id, access_token):
    search_url = "https://emsiservices.com/curricular-skills/courses/search"
    headers = {'Authorization': 'Bearer '+token,'Content-Type': "application/json"}
    # Required Params: site='ccc', isPublished=true, limit <= 100
    search_payload = '{ \"data\": { \"type\": \"courseSearch\", \"attributes\": { \"filter\": { \"site\": { \"in\": [\"ccc\"]}, \"isPublished\": true, \"associatedGroups\": { \"in\": [\"'+group_id+'\"]}}, \"limit\": 100}}}'
    search = requests.post(url=search_url, data=search_payload, headers=headers)
    search = search.json()
    group_count = 0
    r = search['data']
    courses = []
    for course in r:
        cdict = {'id':course['id'], 'course_id':course['attributes']['courseId'], 'title':\
                course['attributes']['title'], 'description':course['attributes']['description'], \
                'skills':course['attributes']['skills'], 'credits':course['attributes']['credits'], \
                'url':course['attributes']['url'],'updated':course['attributes']['updatedAt']}
        courses.append(cdict)
    return(pandas.DataFrame(courses))
    # total available = 1,588

In [494]:
def courses_by_group_type(group_type_id, token):
    gbt = groups_by_type(group_type_id, token)
    all_courses = []
    for group_id in gbt.group_id.unique().tolist():
        courses = courses_by_group(group_id, token)
        courses['group_id'] = group_id
        courses['group'] = gbt[gbt['group_id']==group_id].title.values[0]
        all_courses.append(courses)
    df = pandas.concat(all_courses)
    df = df.reset_index(drop=True)
    df = df[['group','course_id','title','description','credits','skills','updated',\
             'id','group_id', 'url']]
    return(df)

In [ ]:
# Courses by group (by group type)
basic_cert_courses = courses_by_group_type(group_types['Basic Certificate'], token)
adv_cert_courses = courses_by_group_type(group_types['Advanced Certificate'], token)
arts_courses = courses_by_group_type(group_types['Associate in Arts'], token)
applied_science_courses = courses_by_group_type(group_types['Associate in Applied Science'], token)
eng_science_courses = courses_by_group_type(group_types['Associate in Engineering Science'], token)
fine_arts_courses = courses_by_group_type(group_types['Associate in Fine Arts'], token)
science_courses = courses_by_group_type(group_types['Associate in Science'], token)
general_courses = courses_by_group_type(group_types['Associate in General Studies'], token)
basic_cert_courses['group_type'] = 'Basic Certification'
adv_cert_courses['group_type'] = 'Advanced Certification'
arts_courses['group_type'] = 'Associate in Arts'
applied_science_courses['group_type'] = 'Associate in Applied Science'
eng_science_courses['group_type'] = 'Associate in Engineering Science'
fine_arts_courses['group_type'] = 'Associate in Fine Arts'
science_courses['group_type'] = 'Associate in Science'
general_courses['group_type'] = 'Associate in General Studies'
all_courses = pandas.concat([basic_cert_courses,adv_cert_courses,arts_courses,applied_science_courses,\
                            eng_science_courses,fine_arts_courses,science_courses])
all_courses = all_courses.reset_index(drop=True)

### Degrees linked to their courses (with courses linked to their skills)

In [832]:
# df of all degrees (groups)
gbt = pandas.concat([bc, ac, ags, aa, aas, aes, afs, asc])
gbt['course_count'] = gbt['courses'].apply(len)
gbt = gbt.rename(columns={'title':'degree','groupType':'degree_type','groupType_id':'degree_type_id','group_id':'degree_id'})
gbt = gbt[['degree','degree_id','degree_type','description','url','updated','course_count','courses']]
gbt = gbt.reset_index(drop=True)

In [880]:
def stretch_degrees(df):
    out = pandas.DataFrame(columns=df.columns)
    out_rows = 0
    for i in df.index:
        courses = df.loc[i, 'courses']
        count = df.loc[i, 'course_count']
        for index in range(0, count):
            out.loc[out_rows] = df.loc[i]
            out.loc[out_rows, 'course_id'] = courses[index]['id']
            out_rows += 1
    del out['courses']
    return(out)

In [ ]:
degrees = stretch_degrees(gbt) # only 892 classes linked to a degree
courses = df.rename(columns={'description':'course_description','url':'course_url','updated':'course_updated',\
                             'title':'course_title'})
degrees = degrees.merge(courses, how='left', on='course_id')

In [896]:
def stretch_courses(df):
    df['skill_count'] = df['skills'].apply(len)
    out = pandas.DataFrame(columns=df.columns)
    out_rows = 0
    for i in df.index:
        courses = df.loc[i, 'skills']
        skill_count = df.loc[i, 'skill_count']
        for index in range(0, skill_count):
            out.loc[out_rows] = df.loc[i]
            out.loc[out_rows, 'skill_id'] = courses[index]
            out_rows += 1
    del out['skills']
    return(out)

In [866]:
# Degree-course-skill level file
# row for every degree-course-skill pairing
dcs = stretch_courses(degrees)
skills = pandas.read_excel('C:/WDEMP/EMSI/Curricular Skills API/EMSI_skills.xlsx')
dcs = dcs.merge(skills[['id','name','type']] , how='left', left_on='skill_id', right_on='id')
del dcs['id']
dcs.to_excel('C:/WDEMP/EMSI/Curricular Skills API/degrees_courses_skills.xlsx')

In [870]:
# Courses not linked to a degree via group
courses = df.course_id.unique().tolist() # 1,588 total courses
degree_courses = degrees.course_id.unique().tolist() # 892 linked to degrees
unlinked = [x for x in courses if x not in degree_courses] # 696 unlinked to a degree
loose = df[df['course_id'].isin(unlinked)]
ls = stretch_courses(loose)
ls = ls.merge(skills[['id','name','type']] , how='left', left_on='skill_id', right_on='id')
del ls['id']
ls.to_excel('C:/WDEMP/EMSI/Curricular Skills API/loose_courses_skills.xlsx')

### Export degree info

In [775]:
# Export degree program info as all_groups
# Adding groupTypes to group-level df (all_groups)
types = dict(zip(gts['id'], gts['groups']))
gt_names = dict(zip(gts['id'], gts['title']))
for i in all_groups.index:
    gid = all_groups.loc[i, 'group_id']
    for gt in types.keys():
        group_ids = [x['id'] for x in types[gt]]
        if gid in group_ids:
            all_groups.loc[i, 'groupType_id'] = gt
            all_groups.loc[i, 'groupType_title'] = gt_names[gt]
all_groups['course_count'] = all_groups['courses'].apply(len)
all_groups['transfer'] = all_groups['url'].str.contains('transfer')
all_groups['transfer'] = all_groups['transfer'].replace({True:1,False:0})
all_groups = all_groups.rename(columns={'title':'name','groupType_title':'degree','groupTypeClass':'degree_type'})
all_groups = all_groups[['name','degree','degree_type','description','transfer','url','courses','course_count','updated',\
                         'group_id','groupType_id']]
all_groups = all_groups.sort_values(by=['name','degree'])
all_groups = all_groups.reset_index(drop=True)
all_groups.to_excel('C:/WDEMP/EMSI/Curricular Skills API/CCC_degrees.xlsx')

### Courses linked to Skills (no degree info)

In [923]:
ls = ls.rename(columns={'title':'course_title','description':'course_description','url':'course_url',\
                        'updated':'course_updated'})
cols = ls.columns.tolist()
cols = cols[:9]
linked = dcs[cols]
linked = linked.merge(skills[['id','name','type']] , how='left', left_on='skill_id', right_on='id')
courses_skills = pandas.concat([linked, ls])
del courses_skills['id']
courses_skills = courses_skills.reset_index(drop=True)
courses_skills.to_excel('C:/WDEMP/EMSI/Curricular Skills API/courses_skills.xlsx')